# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
!pip install pymongo

In [4]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [12]:
client.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [13]:
db = client.get_database("ironhack")

In [14]:
db.list_collection_names()

['companies']

In [17]:
c = db.get_collection("companies")

In [18]:
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [22]:
Babel = list(c.find({'name':'Babelgum'}))

In [30]:
#Babel

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [36]:
# Your Code --> lo dejo sin printear porque es largo
#list(c.find({"number_of_employees": {"$gte": 5000}}).limit(1))


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [43]:
# Your Code
proyec = {"name": 1, "founded_year": 1, "_id":0}
cond1 = {"founded_year": {"$gte": 2000}}
cond2 = {"founded_year": {"$lte": 2005}}

list(c.find({"$and": [cond1,cond2]}, proyec).limit(3))


[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [53]:
#Valuation Amount --> prueba para entender
#list(c.find({"ipo.valuation_amount":{"$gt": 1}}, {"ipo.valuation_amount": 1}))

In [55]:
proy = {"name":1, "ipo":1, "_id":0}

cond1 = {"ipo.valuation_amount":{"$gt": 100000000}}
cond2 = {"founded_year":{"$lt": 2010}}

companies2 = list(c.find({"$and": [cond1,cond2]},proy))

In [57]:
#companies2 

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [68]:
# Your Code
proy3 = {"name":1 , "number_of_employees":1 , "founded_year":1 , "_id":0}

c1 = {"number_of_employees":{"$lt": 1000}}
c2 = {"founded_year":{"$lt": 2005}}

companies3 =  list(c.find({"$and": [c1,c2]},proy3).sort("number_of_employees",-1).limit(3))
companies3


[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997}]

### 6. All the companies that don't include the `partners` field.

In [86]:
# Your Code
cond_exi = {"partners": {"$exists": False}}
hola = list(c.find(cond_exi))
len(hola) #no hay

0

### 7. All the companies that have a null type of value on the `category_code` field.

In [107]:
companies = list(c.find({"category_code": {"$type": "null" }}))
#companies

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [108]:
# Your Code
proyZ = {"name":1 , "number_of_employees":1 , "_id":0}

co1 = {"number_of_employees":{"$lt": 1000}}
co2 = {"number_of_employees":{"$gt": 100}}

companies8 =  list(c.find({"$and": [co1,co2]},proyZ))
#companies8

### 9. Order all the companies by their IPO price in a descending order.

In [117]:
# Your Code
pro =  {"name":1 , "ipo":1, "number_of_employees":1 , "_id":0}
order = list(c.find({},pro).sort("ipo",-1).limit(2))
order

[{'name': 'GREE',
  'number_of_employees': 700,
  'ipo': {'valuation_amount': 108960000000,
   'valuation_currency_code': 'JPY',
   'pub_year': 2008,
   'pub_month': 12,
   'pub_day': 17,
   'stock_symbol': '3632'}},
 {'name': 'Facebook',
  'number_of_employees': 5299,
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [118]:
# Your Code
p =  {"name":1 , "number_of_employees":1 , "_id":0}
order = list(c.find({},pro).sort("number_of_employees",-1).limit(2))
order

[{'name': 'Siemens',
  'number_of_employees': 405000,
  'ipo': {'valuation_amount': None,
   'valuation_currency_code': 'USD',
   'pub_year': 2001,
   'pub_month': 3,
   'pub_day': 23,
   'stock_symbol': 'NYSE:SI'}},
 {'name': 'IBM',
  'number_of_employees': 388000,
  'ipo': {'valuation_amount': None,
   'valuation_currency_code': 'USD',
   'pub_year': 1978,
   'pub_month': 1,
   'pub_day': 13,
   'stock_symbol': 'NYSE:IBM'}}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [123]:
# Your Code
p =  {"name":1 ,"founded_month":1 , "_id":0}
secondsemester = ({"founded_month":{"$gt": 6}})

second = list(c.find(secondsemester,p).limit(3))
second

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Omnidrive', 'founded_month': 11}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
companies3 =  list(c.find({"$and": [c1,c2]},proy3).sort("number_of_employees",-1).limit(3))

In [137]:
# Your Code

before2000 = ({"founded_year":{"$lt": 2000}})
acq = ({"acquisition.price_amount":{"$gt": 1000000}})


fund = list(c.find({"$and": [before2000, acq]}))
#fund

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [143]:
retrieve = {"name":1 ,"acquisition.price_amount":1 , "founded_year":1, "_id":0}
after2010 = ({"founded_year":{"$gt": 2010}})

trece = list(c.find(after2010, retrieve).sort("acquisition.price_amount",-1).limit(2))
trece
# Your Code

[{'name': 'Mokitown', 'founded_year': 2011},
 {'name': 'CircleUp', 'founded_year': 2011}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [150]:
# Your Code
r14 = {"name":1 , "founded_year":1, "_id":0}
catorce = list(c.find({}, r14).sort("founded_year",-1).limit(15))
catorce

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Tongxue', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Shopseen', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'VistaGen Therapeutics', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'headr', 'founded_year': 2012},
 {'name': 'PeekYou', 'founded_year': 2012}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [157]:
# Your Code
r15 = {"name":1 , "founded_day":1, "_id":0}
dia = ({"founded_day":{"$lt": 8}})

quince = list(c.find(dia,r15).sort("founded_day",-1).limit(3))
quince

[{'name': 'Jive Software', 'founded_day': 7},
 {'name': 'Meetup', 'founded_day': 7},
 {'name': 'Google', 'founded_day': 7}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [166]:
# Your Code
colu = {"name":1 , "category_code":1, "number_of_employees":1, "_id":0}

web = {"category_code": {"$in": ["web"]}}
cuatromil = ({"number_of_employees":{"$gt": 4000}})

oye = list(c.find({"$and":[web,cuatromil]}, colu).sort("number_of_employees",1).limit(3))
oye

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [171]:
# Your Code
coluS = {"name":1 , "acquisition.price_amount":1, "acquisition.price_currency_code":1, "_id":0}

more10 = {"acquisition.price_amount": {"$gt": 10000000}}
currency = {"acquisition.price_currency_code": {"$in": ["EUR"]}}

DIECISIETE = list(c.find({"$and":[more10,currency]}, coluS).limit(3))
DIECISIETE

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [173]:
# Your Code
columm = {"name":1 , "acquisition":1, "founded_month":1 ,"_id":0}
FIRST = ({"founded_month":{"$lt": 4}})

dieciocho = list(c.find(FIRST, columm).limit(1))
dieciocho

[{'name': 'StumbleUpon',
  'founded_month': 2,
  'acquisition': {'price_amount': 29000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2009/04/13/ebay-unacquires-stumbleupon/',
   'source_description': "StumbleUpon Beats Skype In Escaping EBay's Clutches",
   'acquired_year': 2009,
   'acquired_month': 4,
   'acquired_day': None,
   'acquiring_company': {'name': 'StumbleUpon', 'permalink': 'stumbleupon'}}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code



### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code